<a href="https://colab.research.google.com/github/ejyepezm/PPIA/blob/main/unidad_2_paradigmas_avanzados/4_Declarativo_SQL_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 💊 Cápsula 4: El Comandante de Datos
**Tema:** Paradigma Declarativo, SQL y Pandas "Method Chaining".

## 1. Deja de micro-gestionar tus datos

El paradigma **Declarativo** es como ser el jefe: das órdenes de **QUÉ** quieres, no de **CÓMO** hacerlo.
*   **Imperativo (Micro-management):** "Crea variable X, abre bucle, suma 1, si es mayor a 5 guarda..."
*   **Declarativo (El Jefe):** "Dame el promedio de ventas por región filtrando los nulos".

### SQL: El Rey Declarativo
SQL es el ejemplo perfecto. `SELECT * FROM ventas WHERE total > 100`. No le dices a la base de datos cómo buscar en el disco duro, solo pides el resultado.

### Pandas Method Chaining
En Python, podemos imitar esto encadenando métodos. En lugar de llenar la memoria con variables temporales (`df1`, `df2`, `df_final`), escribimos una sola "frase" de transformaciones.

In [1]:
import pandas as pd
import sqlite3

# --- PREPARACIÓN DE DATOS (No tocar) ---
# Creamos un DataFrame simple
data = {
    'producto': ['Manzana', 'Pera', 'Manzana', 'Uva', 'Pera', 'Manzana'],
    'region': ['Norte', 'Norte', 'Sur', 'Sur', 'Norte', 'Oeste'],
    'ventas': [100, 50, 120, 200, 60, 50],
    'calidad': ['A', 'B', 'A', 'A', 'B', 'C'] # Dato extra
}
df = pd.DataFrame(data)

# --- 1. MODO SQL (Declarativo Puro) ---
# Creamos una mini DB en memoria RAM
conn = sqlite3.connect(':memory:')
df.to_sql('tabla_ventas', conn, index=False)

query = """
SELECT region, AVG(ventas) as promedio
FROM tabla_ventas
WHERE ventas > 50
GROUP BY region
ORDER BY promedio DESC
"""
resultado_sql = pd.read_sql(query, conn)
print("--- Resultado SQL ---")
print(resultado_sql)

# --- 2. MODO PANDAS DECLARATIVO (Method Chaining) ---
# Fíjate cómo se lee como una historia de arriba a abajo
resultado_pandas = (
    df
    .query('ventas > 50')               # WHERE
    .groupby('region')                  # GROUP BY
    ['ventas'].mean()                   # AVG (Select)
    .sort_values(ascending=False)       # ORDER BY
)

print("\n--- Resultado Pandas Chaining ---")
print(resultado_pandas)

--- Resultado SQL ---
  region  promedio
0    Sur     160.0
1  Norte      80.0

--- Resultado Pandas Chaining ---
region
Sur      160.0
Norte     80.0
Name: ventas, dtype: float64


## 🔥 Micro-Desafío: El Pipeline Limpio

Tienes un DataFrame de empleados desordenado. Tu jefe quiere un reporte rápido, pero odia el "código espagueti" lleno de variables `df1`, `df2`, `df3`.

**Tu Misión:**
Escribe **UNA SOLA CADENA** de métodos de Pandas (usando paréntesis `(...)` para saltar líneas) que haga lo siguiente en orden:
1.  Filtre a los empleados del departamento 'Ventas' (`.query`).
2.  Cree una nueva columna 'bono' que sea el 10% del salario (`.assign`).
3.  Seleccione solo las columnas 'nombre', 'salario' y 'bono' (`[['col1', ...]]` o `.loc`).
4.  Ordene por 'bono' de mayor a menor (`.sort_values`).

*Tip: El código está medio montado, solo debes llenar los huecos.*

In [ ]:
# Datos de empleados
empleados = pd.DataFrame({
    'nombre': ['Ana', 'Luis', 'Pedro', 'Marta', 'Sofía'],
    'depto': ['Ventas', 'IT', 'Ventas', 'HR', 'Ventas'],
    'salario': [2000, 3000, 1500, 2500, 4000]
})

print("Datos Originales:")
print(empleados)

# --- TU CÓDIGO AQUÍ ---
reporte_final = (
    empleados
    .query("depto == 'Ventas'")                 # 1. Filtrar Ventas
    .assign(bono=lambda x: x['salario'] * 0.10) # 2. Calcular Bono (10%)
    [['nombre', 'salario', 'bono']]             # 3. Seleccionar columnas
    # TODO: Agrega el paso 4 aquí para ordenar por 'bono' descendente
    # .sort_values(...)
)

print("\n--- Reporte Final (Debe ser: Sofía, Ana, Pedro) ---")
print(reporte_final)

# Validación simple
top_empleado = reporte_final.iloc[0]['nombre']
if top_empleado == 'Sofía':
    print("\n✅ ¡Correcto! Has creado un pipeline declarativo.")
else:
    print(f"\n❌ Algo falló. El empleado top debería ser Sofía, pero es {top_empleado}.")